In [2]:
import torch
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import Coauthor
from torch_geometric.datasets import Amazon
from torch_geometric.datasets import Actor,AttributedGraphDataset,HeterophilousGraphDataset,Planetoid,WikipediaNetwork,Coauthor
from torch_geometric.nn import GAT,GIN
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
import torch_geometric
import torch
from torch_geometric.nn import Linear,GCNConv,SuperGATConv,GATConv,GeneralConv,SGConv,FAConv,EGConv,GCN2Conv,SSGConv,GENConv
from torch_geometric.utils import degree

In [4]:
dsname="Physics"

if dsname in ["CS","Physics","BlogCatalog","WIKI","cora_ml","citeseer","pubmed","Photo","Computers" ]:
    if dsname =="CS":
        nm = Coauthor(root="./datasets2",name=dsname)
    if dsname =="Physics":
        nm = Coauthor(root="./datasets2",name=dsname)
    if dsname =="BlogCatalog":
        nm=AttributedGraphDataset(root="./datasets",name=dsname)
    if dsname =="WIKI":
        nm=AttributedGraphDataset(root="./datasets",name=dsname)
    if dsname =="cora_ml":
        nm = CitationFull(root="./datasets",name=dsname)
    if dsname =="citeseer":
        nm = CitationFull(root="./datasets",name=dsname)
    if dsname =="pubmed":
        nm = CitationFull(root="./datasets",name=dsname)
    if dsname =="Photo":
        nm = Amazon(root="./datasets",name=dsname)
    if dsname =="Computers":
        nm = Amazon(root="./datasets",name=dsname)
        
    print(f'Dataset: {nm}:')
    print(f'Number of graphs: {len(nm)}')
    print(f'Number of features: {nm.num_features}')
    print(f'Number of classes: {nm.num_classes}')
    print(nm.x.shape)
    print(nm.edge_index.shape)
    node_degree=degree(nm[0].edge_index[0])
    avgdegree=node_degree.mean().item()
    print(f'Avg degree: {avgdegree}')
    
else:
    print("The name of datsets error,try again!")



Dataset: CoauthorPhysics():
Number of graphs: 1
Number of features: 8415
Number of classes: 5
torch.Size([34493, 8415])
torch.Size([2, 495924])
Avg degree: 14.37752628326416


### GCN_2L

In [ ]:
def mynorm(tensor):
    # 计算每一行的最小值和最大值
    min_values = tensor.min(dim=1, keepdim=True)[0]
    max_values = tensor.max(dim=1, keepdim=True)[0]
    # 进行最小-最大归一化
#     normalized_tensor = (tensor - min_values) / (max_values - min_values + 1e-8)               ### 0-1
    normalized_tensor = 2 * (tensor - min_values) / (max_values - min_values + 1e-8) - 1       ### -1-1
    return normalized_tensor


class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.fc1 = nn.Linear(in_channels,64)
        self.conv1 = GCNConv(64,32)
        self.conv2 = GCNConv(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.fc1(x)
        x1 = self.conv1(x0, edge_index).relu()         ##
        x2 = self.conv2(x1, edge_index).softmax(dim=-1)
 
        return x2      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)
#     print(label.to(torch.float32))
#     print(label.dtype)
#     print(out.dtype)
    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
            model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
#     pd.DataFrame(framedata).to_csv("./GCN/GCN_{}_2L.csv".format(dsname))

2025-11-05 17:00:53.656109: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 17:00:54.657862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 17:00:57.158473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/root/anaconda3/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:90: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for bett

Epoch: 0, Loss: 1.6084, Train: 0.6510,Test: 0.6490
Epoch: 1, Loss: 1.5045, Train: 0.6954,Test: 0.7023
Epoch: 2, Loss: 1.3555, Train: 0.7378,Test: 0.7426
Epoch: 3, Loss: 1.2434, Train: 0.7991,Test: 0.8006
Epoch: 4, Loss: 1.1678, Train: 0.8314,Test: 0.8319
Epoch: 5, Loss: 1.1149, Train: 0.8565,Test: 0.8586
Epoch: 6, Loss: 1.0808, Train: 0.8826,Test: 0.8829
Epoch: 7, Loss: 1.0564, Train: 0.9032,Test: 0.9093
Epoch: 8, Loss: 1.0351, Train: 0.9173,Test: 0.9220
Epoch: 9, Loss: 1.0162, Train: 0.9280,Test: 0.9296
Epoch: 10, Loss: 1.0006, Train: 0.9341,Test: 0.9345
Epoch: 11, Loss: 0.9887, Train: 0.9394,Test: 0.9432
Epoch: 12, Loss: 0.9801, Train: 0.9449,Test: 0.9467
Epoch: 13, Loss: 0.9742, Train: 0.9477,Test: 0.9493
Epoch: 14, Loss: 0.9706, Train: 0.9488,Test: 0.9493
Epoch: 15, Loss: 0.9685, Train: 0.9495,Test: 0.9496
Epoch: 16, Loss: 0.9667, Train: 0.9495,Test: 0.9487
Epoch: 17, Loss: 0.9645, Train: 0.9506,Test: 0.9507
Epoch: 18, Loss: 0.9618, Train: 0.9520,Test: 0.9522
Epoch: 19, Loss: 0.959

Epoch: 61, Loss: 0.9464, Train: 0.9653,Test: 0.9632
Epoch: 62, Loss: 0.9463, Train: 0.9653,Test: 0.9635
Epoch: 63, Loss: 0.9462, Train: 0.9655,Test: 0.9635
Epoch: 64, Loss: 0.9461, Train: 0.9658,Test: 0.9632
Epoch: 65, Loss: 0.9460, Train: 0.9659,Test: 0.9632
Epoch: 66, Loss: 0.9459, Train: 0.9662,Test: 0.9629
Epoch: 67, Loss: 0.9458, Train: 0.9662,Test: 0.9629
Epoch: 68, Loss: 0.9457, Train: 0.9662,Test: 0.9629
Epoch: 69, Loss: 0.9456, Train: 0.9662,Test: 0.9629
Epoch: 70, Loss: 0.9455, Train: 0.9664,Test: 0.9632
Epoch: 71, Loss: 0.9454, Train: 0.9666,Test: 0.9632
Epoch: 72, Loss: 0.9453, Train: 0.9669,Test: 0.9632
Epoch: 73, Loss: 0.9451, Train: 0.9669,Test: 0.9632
Epoch: 74, Loss: 0.9450, Train: 0.9671,Test: 0.9635
Epoch: 75, Loss: 0.9448, Train: 0.9672,Test: 0.9635
Epoch: 76, Loss: 0.9446, Train: 0.9672,Test: 0.9632
Epoch: 77, Loss: 0.9445, Train: 0.9674,Test: 0.9632
Epoch: 78, Loss: 0.9444, Train: 0.9674,Test: 0.9638
Epoch: 79, Loss: 0.9443, Train: 0.9674,Test: 0.9638
Epoch: 80, L

Epoch: 21, Loss: 0.9576, Train: 0.9544,Test: 0.9551
Epoch: 22, Loss: 0.9555, Train: 0.9549,Test: 0.9571
Epoch: 23, Loss: 0.9544, Train: 0.9549,Test: 0.9591
Epoch: 24, Loss: 0.9539, Train: 0.9550,Test: 0.9600
Epoch: 25, Loss: 0.9536, Train: 0.9551,Test: 0.9594
Epoch: 26, Loss: 0.9532, Train: 0.9559,Test: 0.9603
Epoch: 27, Loss: 0.9526, Train: 0.9565,Test: 0.9600
Epoch: 28, Loss: 0.9519, Train: 0.9576,Test: 0.9612
Epoch: 29, Loss: 0.9511, Train: 0.9580,Test: 0.9612
Epoch: 30, Loss: 0.9504, Train: 0.9582,Test: 0.9620
Epoch: 31, Loss: 0.9499, Train: 0.9584,Test: 0.9614
Epoch: 32, Loss: 0.9496, Train: 0.9589,Test: 0.9612
Epoch: 33, Loss: 0.9494, Train: 0.9594,Test: 0.9600
Epoch: 34, Loss: 0.9493, Train: 0.9596,Test: 0.9597
Epoch: 35, Loss: 0.9492, Train: 0.9596,Test: 0.9591
Epoch: 36, Loss: 0.9491, Train: 0.9601,Test: 0.9588
Epoch: 37, Loss: 0.9489, Train: 0.9605,Test: 0.9594
Epoch: 38, Loss: 0.9487, Train: 0.9610,Test: 0.9603
Epoch: 39, Loss: 0.9485, Train: 0.9614,Test: 0.9603
Epoch: 40, L

Epoch: 79, Loss: 0.9440, Train: 0.9674,Test: 0.9643
Epoch: 80, Loss: 0.9438, Train: 0.9676,Test: 0.9649
Epoch: 81, Loss: 0.9437, Train: 0.9677,Test: 0.9652
Epoch: 82, Loss: 0.9435, Train: 0.9677,Test: 0.9649
Epoch: 83, Loss: 0.9434, Train: 0.9679,Test: 0.9649
Epoch: 84, Loss: 0.9433, Train: 0.9679,Test: 0.9652
Epoch: 85, Loss: 0.9431, Train: 0.9681,Test: 0.9649
Epoch: 86, Loss: 0.9430, Train: 0.9683,Test: 0.9649
Epoch: 87, Loss: 0.9429, Train: 0.9683,Test: 0.9655
Epoch: 88, Loss: 0.9427, Train: 0.9685,Test: 0.9655
Epoch: 89, Loss: 0.9426, Train: 0.9688,Test: 0.9652
Epoch: 90, Loss: 0.9425, Train: 0.9690,Test: 0.9652
Epoch: 91, Loss: 0.9423, Train: 0.9690,Test: 0.9652
Epoch: 92, Loss: 0.9422, Train: 0.9690,Test: 0.9655
Epoch: 93, Loss: 0.9421, Train: 0.9691,Test: 0.9655
Epoch: 94, Loss: 0.9419, Train: 0.9691,Test: 0.9655
Epoch: 95, Loss: 0.9418, Train: 0.9695,Test: 0.9652
Epoch: 96, Loss: 0.9417, Train: 0.9697,Test: 0.9649
Epoch: 97, Loss: 0.9416, Train: 0.9698,Test: 0.9649
Epoch: 98, L

Epoch: 35, Loss: 0.9499, Train: 0.9591,Test: 0.9597
Epoch: 36, Loss: 0.9497, Train: 0.9594,Test: 0.9600
Epoch: 37, Loss: 0.9494, Train: 0.9598,Test: 0.9609
Epoch: 38, Loss: 0.9491, Train: 0.9606,Test: 0.9617
Epoch: 39, Loss: 0.9488, Train: 0.9605,Test: 0.9620
Epoch: 40, Loss: 0.9485, Train: 0.9607,Test: 0.9635
Epoch: 41, Loss: 0.9484, Train: 0.9607,Test: 0.9626
Epoch: 42, Loss: 0.9483, Train: 0.9607,Test: 0.9629
Epoch: 43, Loss: 0.9482, Train: 0.9607,Test: 0.9623
Epoch: 44, Loss: 0.9482, Train: 0.9610,Test: 0.9623
Epoch: 45, Loss: 0.9480, Train: 0.9615,Test: 0.9620
Epoch: 46, Loss: 0.9479, Train: 0.9617,Test: 0.9620
Epoch: 47, Loss: 0.9477, Train: 0.9619,Test: 0.9623
Epoch: 48, Loss: 0.9476, Train: 0.9620,Test: 0.9617
Epoch: 49, Loss: 0.9476, Train: 0.9623,Test: 0.9623
Epoch: 50, Loss: 0.9476, Train: 0.9625,Test: 0.9620
Epoch: 51, Loss: 0.9475, Train: 0.9630,Test: 0.9629
Epoch: 52, Loss: 0.9474, Train: 0.9631,Test: 0.9632
Epoch: 53, Loss: 0.9474, Train: 0.9634,Test: 0.9629
Epoch: 54, L

Epoch: 95, Loss: 0.9423, Train: 0.9689,Test: 0.9649
Epoch: 96, Loss: 0.9422, Train: 0.9690,Test: 0.9646
Epoch: 97, Loss: 0.9421, Train: 0.9690,Test: 0.9646
Epoch: 98, Loss: 0.9420, Train: 0.9692,Test: 0.9655
Epoch: 99, Loss: 0.9419, Train: 0.9693,Test: 0.9655
[0.1, 98, 0.9419993162155151, 0.9691717939632123, 0.9655072463768116]
Epoch: 0, Loss: 1.6099, Train: 0.7650,Test: 0.7681
Epoch: 1, Loss: 1.5430, Train: 0.7807,Test: 0.7786
Epoch: 2, Loss: 1.4162, Train: 0.7875,Test: 0.7893
Epoch: 3, Loss: 1.2914, Train: 0.8018,Test: 0.8026
Epoch: 4, Loss: 1.1995, Train: 0.8365,Test: 0.8388
Epoch: 5, Loss: 1.1377, Train: 0.8781,Test: 0.8800
Epoch: 6, Loss: 1.0901, Train: 0.9047,Test: 0.9113
Epoch: 7, Loss: 1.0519, Train: 0.9215,Test: 0.9258
Epoch: 8, Loss: 1.0237, Train: 0.9332,Test: 0.9330
Epoch: 9, Loss: 1.0043, Train: 0.9388,Test: 0.9429
Epoch: 10, Loss: 0.9915, Train: 0.9441,Test: 0.9467
Epoch: 11, Loss: 0.9832, Train: 0.9474,Test: 0.9493
Epoch: 12, Loss: 0.9774, Train: 0.9483,Test: 0.9499
Epoc

### GAT——2L

In [ ]:


def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_set_ind], data.y[train_label_ind])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)

    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    index=[i for i in range(dataset.x.shape[0])]
    train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1,random_state=42,stratify=dataset.y)
            


    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)
            value=[0.1*percent,0,0,0,0]
            
                    # 初始化 GAT 模型
            model = GAT(in_channels=dataset.num_node_features,
                        hidden_channels=32,
                        num_layers=2,
                        out_channels=dataset.num_classes).to(device)
        #     model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                pass
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./GAT/GAT_{}_2L.csv".format(dsname))    
    

### GIN-2L

In [ ]:


def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_set_ind], data.y[train_label_ind])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)

    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    

    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)
            value=[0.1*percent,0,0,0,0]
            
                    # 初始化 GAT 模型
            model = GIN(in_channels=dataset.num_node_features,
                        hidden_channels=32,
                        num_layers=2,
                        out_channels=dataset.num_classes).to(device)
        #     model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                pass
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./GIN/GIN_{}_2L.csv".format(dsname))    
    

### GCNII-5L

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels,num_heads=1):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = GCN2Conv(32,0.1)
        self.conv2 = GCN2Conv(32,0.1)
        self.conv3 = GCN2Conv(32,0.1)
        self.conv4 = GCN2Conv(32,0.1)
        self.conv5 = GCN2Conv(32,0.1)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,x0,edge_index).relu()
        x2 = self.conv2(x1,x0,edge_index).relu()
        x3 = self.conv3(x2,x0,edge_index).relu()
        x4 = self.conv4(x3,x0,edge_index).relu()
        x5 = self.conv5(x4,x0,edge_index).relu()
        x6 = self.conv6(x5)
 
        return x6      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)
#     print(label.to(torch.float32))
#     print(label.dtype)
#     print(out.dtype)
    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
            model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./GCNII/GCNII_{}_5L.csv".format(dsname))
    

#### GAT2--5l 

In [ ]:


def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_set_ind], data.y[train_label_ind])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)

    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    index=[i for i in range(dataset.x.shape[0])]
    train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1,random_state=42,stratify=dataset.y)
            


    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)
            value=[0.1*percent,0,0,0,0]
            
                    # 初始化 GAT 模型
            model = GAT(in_channels=dataset.num_node_features,
                        hidden_channels=32,
                        num_layers=5,
                        out_channels=dataset.num_classes,
                        v2=True).to(device)
        #     model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                pass
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./GAT2/GAT2_{}_5L.csv".format(dsname))    
    

#### FAGCN--5L

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels,num_heads=1):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = FAConv(32,0.3,0.4)
        self.conv2 = FAConv(32,0.3,0.4)
        self.conv3 = FAConv(32,0.3,0.4)
        self.conv4 = FAConv(32,0.3,0.4)
        self.conv5 = FAConv(32,0.3,0.4)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,x0,edge_index).relu()
        x2 = self.conv2(x1,x0,edge_index).relu()
        x3 = self.conv3(x2,x0,edge_index).relu()
        x4 = self.conv4(x3,x0,edge_index).relu()
        x5 = self.conv5(x4,x0,edge_index).relu()
        x6 = self.conv6(x5)
 
        return x6      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)
#     print(label.to(torch.float32))
#     print(label.dtype)
#     print(out.dtype)
    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
            model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./FAGCN/FAGCN_{}_5L.csv".format(dsname))
    

### SSGCN--5L

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = SSGConv(32,32,0.05)
        self.conv2 = SSGConv(32,32,0.05)
        self.conv3 = SSGConv(32,32,0.05)
        self.conv4 = SSGConv(32,32,0.05)
        self.conv5 = SSGConv(32,32,0.05)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,edge_index).relu()
        x2 = self.conv2(x1,edge_index).relu()
        x3 = self.conv3(x2,edge_index).relu()
        x4 = self.conv4(x3,edge_index).relu()
        x5 = self.conv5(x4,edge_index).relu()
        x6 = self.conv6(x5)   ## 1hop       inf
 
        return x6         

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)

    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
            model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./SSGCN/SSGCN_{}_5L.csv".format(dsname))
    

### generalGCN_5

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = GeneralConv(32,32)
        self.conv2 = GeneralConv(32,32)
        self.conv3 = GeneralConv(32,32)
        self.conv4 = GeneralConv(32,32)
        self.conv5 = GeneralConv(32,32)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,edge_index).relu()
        x2 = self.conv2(x1,edge_index).relu()
        x3 = self.conv3(x2,edge_index).relu()
        x4 = self.conv4(x3,edge_index).relu()
        x5 = self.conv5(x4,edge_index).relu()
        x6 = self.conv6(x5)   ## 1hop       inf
 
        return x6      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)

    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
#             model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./generalGCN/gGCN_{}_5L.csv".format(dsname))
    

### superGAT_5

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = SuperGATConv(32,32)
        self.conv2 = SuperGATConv(32,32)
        self.conv3 = SuperGATConv(32,32)
        self.conv4 = SuperGATConv(32,32)
        self.conv5 = SuperGATConv(32,32)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,edge_index).relu()
        x2 = self.conv2(x1,edge_index).relu()
        x3 = self.conv3(x2,edge_index).relu()
        x4 = self.conv4(x3,edge_index).relu()
        x5 = self.conv5(x4,edge_index).relu()
        x6 = self.conv6(x5)   ## 1hop       inf
 
        return x6      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)

    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
        #             model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
            del model
            del optimizer
    pd.DataFrame(framedata).to_csv("./superGAT/superGAT_{}_5L.csv".format(dsname))
    

### GEN—5L

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = GENConv(32,32)
        self.conv2 = GENConv(32,32)
        self.conv3 = GENConv(32,32)
        self.conv4 = GENConv(32,32)
        self.conv5 = GENConv(32,32)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,edge_index).relu()
        x2 = self.conv2(x1,edge_index).relu()
        x3 = self.conv3(x2,edge_index).relu()
        x4 = self.conv4(x3,edge_index).relu()
        x5 = self.conv5(x4,edge_index).relu()
        x6 = self.conv6(x5)   ## 1hop       inf
 
        return x6      

def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)

    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
#             model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./GEN/GEN_{}_5L.csv".format(dsname))
    

#### PMLP——5L

In [ ]:


def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_set_ind], data.y[train_label_ind])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)

    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    index=[i for i in range(dataset.x.shape[0])]
    train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1,random_state=42,stratify=dataset.y)
            


    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)
            value=[0.1*percent,0,0,0,0]
                                    
            model = PMLP(in_channels=dataset.num_node_features,
                        hidden_channels=32,
                        out_channels=dataset.num_classes,
                        num_layers=5).to(device)                                    
                                                 
        #     model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                pass
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./PMLP/PMLP_{}_5L.csv".format(dsname))    
    

### SG--5L

In [ ]:
class SG(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
#         self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv0 = Linear(in_channels,32)
        self.conv1 = SGConv(32,32)
        self.conv2 = SGConv(32,32)
        self.conv3 = SGConv(32,32)
        self.conv4 = SGConv(32,32)
        self.conv5 = SGConv(32,32)
        self.conv6 = Linear(32,out_channels)
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()   
            
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0 = self.conv0(x)   ## 1hop       inf
        x1 = self.conv1(x0,edge_index).relu()
        x2 = self.conv2(x1,edge_index).relu()
        x3 = self.conv3(x2,edge_index).relu()
        x4 = self.conv4(x3,edge_index).relu()
        x5 = self.conv5(x4,edge_index).relu()
        x6 = self.conv6(x5)   ## 1hop       inf
        return x6  
    
    
def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    label=torch.nn.functional.one_hot(data.y[train_label_ind], num_classes=nm.num_classes)
    label=label.to(torch.float32)

    loss = F.cross_entropy(out[train_set_ind], label)
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)
    
    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    
    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)

            value=[0.1*percent,0,0,0,0]
            
            model = SG(dataset.num_features, dataset.num_classes).to(device)
#             model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("./SG/SG_{}_5L.csv".format(dsname))   


Epoch: 0, Loss: 1.6474, Train: 0.1464,Test: 0.1467
Epoch: 1, Loss: 1.6274, Train: 0.1492,Test: 0.1499
Epoch: 2, Loss: 1.6082, Train: 0.3823,Test: 0.3870
Epoch: 3, Loss: 1.5874, Train: 0.5052,Test: 0.5052
Epoch: 4, Loss: 1.5632, Train: 0.5052,Test: 0.5052
Epoch: 5, Loss: 1.5335, Train: 0.5052,Test: 0.5052
Epoch: 6, Loss: 1.4941, Train: 0.5052,Test: 0.5052
Epoch: 7, Loss: 1.4386, Train: 0.5052,Test: 0.5052
Epoch: 8, Loss: 1.3634, Train: 0.5052,Test: 0.5052
Epoch: 9, Loss: 1.2777, Train: 0.5052,Test: 0.5052
Epoch: 10, Loss: 1.2156, Train: 0.5052,Test: 0.5052
Epoch: 11, Loss: 1.2149, Train: 0.5052,Test: 0.5052
Epoch: 12, Loss: 1.2043, Train: 0.5052,Test: 0.5052
Epoch: 13, Loss: 1.1496, Train: 0.5052,Test: 0.5052
Epoch: 14, Loss: 1.0938, Train: 0.5052,Test: 0.5052
Epoch: 15, Loss: 1.0622, Train: 0.5052,Test: 0.5052
Epoch: 16, Loss: 1.0474, Train: 0.5052,Test: 0.5052
Epoch: 17, Loss: 1.0344, Train: 0.5052,Test: 0.5052
Epoch: 18, Loss: 1.0167, Train: 0.5052,Test: 0.5052
Epoch: 19, Loss: 0.995

### SLD-9L

In [ ]:
def mynorm(tensor):
    # 计算每一行的最小值和最大值
    min_values = tensor.min(dim=1, keepdim=True)[0]
    max_values = tensor.max(dim=1, keepdim=True)[0]
    # 进行最小-最大归一化
#     normalized_tensor = (tensor - min_values) / (max_values - min_values + 1e-8)               ### 0-1
    normalized_tensor = 2 * (tensor - min_values) / (max_values - min_values + 1e-8) - 1       ### -1-1
    return normalized_tensor


class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.fc1 = nn.Linear(in_channels,32)
#         self.fc2=nn.Linear(64,16)
        self.conv1 = GCNConv(32,32)
        self.conv2 = GCNConv(32,32)
        self.conv3 = GCNConv(32,32)
        self.conv4 = GCNConv(32,32)
        self.conv5 = GCNConv(32,32)
        self.conv6 = GCNConv(32,32)
        self.conv7 = GCNConv(32,32)
        self.conv8 = GCNConv(32,32)
        
        
#         self.conv7 = GCNConv(16*6,out_channels)
        self.conv9 = nn.Linear(32*9,out_channels)
    
    def reset_parameters(self):
        for layer in self.children():
            layer.reset_parameters()    
    
    
    def forward(self, x, edge_index):
#         print("conv00:{}".format(x))
        x0= self.fc1(x).relu()
        x0=mynorm(x0)
        x1 = self.conv1(x0, edge_index)         ## 1hop       inf
        xx1= x1
        x2 = self.conv2(x1, edge_index)         ## 0,2hop       inf
        xx2=mynorm(x2)- mynorm(x0)
        x3 = self.conv3(x2, edge_index)          ## 1,3hop     inf
        xx3=mynorm(x3)-mynorm(x1)
        
        x4 = self.conv4(x3, edge_index)         ## 0,2,4hop   inf
        xx4=mynorm(x4)-mynorm(x2) 
        
        x5 = self.conv5(x4, edge_index)          ## 1,3,5hop   inf
        xx5=mynorm(x5)-mynorm(x3)
        
        x6 = self.conv6(x5, edge_index)         ## 0,2,4,6hop inf
        xx6=mynorm(x6)-mynorm(x4)

        
        x7 = self.conv7(x6, edge_index)          ## 1,3,5,7 hop   inf
        xx7=mynorm(x7)-mynorm(x5)
        
        x8 = self.conv8(x7, edge_index)         ## 0,2,4,6,8 hop inf
        xx8=mynorm(x8)-mynorm(x6)
        
        
        xx9=torch.cat((x0,xx1,xx2,xx3,xx4,xx5,xx6,xx7,xx8),dim=1)
        x9 = self.conv9(xx9) 
        return x9       


def train(model,train_set_ind,train_label_ind):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_set_ind], data.y[train_label_ind])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test(model,train_set_ind,test_set_ind):
    model.eval()
    ret = model(data.x, data.edge_index)
    pred=ret.argmax(dim=-1)

    mask = train_set_ind
    trainacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
    mask = test_set_ind
    testacc=int((pred[mask] == data.y[mask]).sum()) / int(len(mask))
        
    return trainacc,testacc

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     dataset = CitationFull("./datasets", name="citeseer", transform=T.NormalizeFeatures())
    dataset = nm
    data = dataset[0].to(device)
    

    framedata=[]
    for percent in range(1,10):
        for num in range(10):

            index=[i for i in range(dataset.x.shape[0])]
            train_set_ind,test_set_ind,train_label_ind,test_label_ind=train_test_split(index,index,test_size=0.1*percent,random_state=42,stratify=dataset.y)
            value=[0.1*percent,0,0,0,0]
            
            model = GCN(dataset.num_features, dataset.num_classes).to(device)
            model = torch_geometric.compile(model)                   # Compile the model into an optimized version:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
            
            
            for epoch in range(0, 400):
                loss = train(model,train_set_ind,train_label_ind)
                    # 记录当前权重

                train_acc,test_acc = test(model,train_set_ind,test_set_ind)
                print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_acc:.4f},'f'Test: {test_acc:.4f}')
                if test_acc > value[4]:
                    value[4] = test_acc 
                    value[1] = epoch
                    value[2] = loss
                    value[3] = train_acc
            print(value)
            framedata.append(value)
            time.sleep(0.5)
    pd.DataFrame(framedata).to_csv("CS_9L.csv")
    